In [24]:
from langchain_gigachat import GigaChat

In [25]:
import os
from dotenv import load_dotenv

load_dotenv()

giga_lite = GigaChat(
    credentials=os.getenv("GIGACHAT_APIKEY"),
    scope=os.getenv("GIGACHAT_SCOPE"),
    model="gigachat:latest",
    profanity_check=False,
    verify_ssl_certs=False,
)

giga_max = GigaChat(
    credentials=os.getenv("GIGACHAT_APIKEY"),
    scope=os.getenv("GIGACHAT_SCOPE"),
    model="GigaChat-Max",
    profanity_check=False,
    verify_ssl_certs=False,
)

In [26]:
SYSTEM_PROMPT = """Ты - агент планировщик поисковых запросов.
Твоя задача - анализировать задачу пользователя и разбить её на посиковые запросы для дпльнейшего исследования,
которое будет проводить другой агент.

Процесс работы
1. Анализ исходного запроса:
 - Определи основную тему и подтемы
 - Выявии ключевые сущности (персоны, организации, концепции)
 - Определи временные рамки (если важны)
 - Определи географический контекст (если нужен)
 - Определи тип информации (новости, исследования, мнения, данные)
 - Определи косвенные факторы влиающие на предмет исследования.

2. Генерация поисковых запросов:
 - Создай 3-6 отдельных поисковых запросов
 - Каждый запрос должен быть самодостаточным и конкретным
 - Используй естественный язык, но включай ключевые слова
 - Учитывай возможные синонимы и альтернативные формулировки
 - Разделяй мульти-аспектные запросы на отдельные направления

3. Принципы качества запросов:
 - Конкретность: избегай общих фраз
 - Релевантность: каждый запрос должен освещать отдельный аспект
 - Полнота: совокупность запросов должна покрывать все аспекты исходного вопроса
 - Эффективность: запросы должны быть сформулированы для максимальной релевантности в поисковых системах

Запрос пользователя:
{user_prompt}

Выведи только следующую информацию в формате JSON:
{format_instructions}
"""

In [27]:
USER_PROMPT = """**ИНФОРМАЦИЯ О МОЕЙ КОМПАНИИ:**

Компания «ДИО-Консалт» специализируется на внедрении комплексных корпоративных решений,
предоставляя услуги по автоматизации бизнес-процессов на основе программных продуктов «1С»,
независимо от сферы деятельности и от масштабов самого предприятия.

**Мой запрос:** Планируются переговоры с авиа-компанией 'Победа' (это наш потенциальный клиент),
мне нужно получить ключевые аргументы за то чтобы выбрали имено нас с побробным обоснованием.
"""

In [28]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


class QueryPlannerResponse(BaseModel):
  original_query: str = Field(..., description="Исходный запрос пользователя")
  main_topic: str = Field(..., description="Основная тема")
  key_entities: list[str] = Field(
      default_factory=list, description="Ключевые сущности", examples=["сущность1", "сущность2"]
  )
  subtopics: list[str] = Field(default_factory=list, description="Подтемы")
  time_frame: str = Field(..., description="Актуальность/период")
  info_type: str = Field(..., description="Новости/исследования/мнения")
  search_strategy: str = Field(..., description="Краткое описание стратегии поиска")
  search_queries: list[str] = Field(default_factory=list, description="Поисковые запросы")


parser = PydanticOutputParser(pydantic_object=QueryPlannerResponse)
query_planner_chain = (
    ChatPromptTemplate
    .from_messages([("system", SYSTEM_PROMPT)])
    .partial(format_instructions=parser.get_format_instructions())
    | giga_max
    | parser
)

In [29]:
response = query_planner_chain.invoke({"user_prompt": USER_PROMPT})

response

QueryPlannerResponse(original_query="Планируются переговоры с авиа-компанией 'Победа', мне нужно получить ключевые аргументы за то чтобы выбрали именно нашу компанию 'ДИО-Консалт'.", main_topic="Подготовка аргументов для переговоров с авиакомпанией 'Победа'", key_entities=['ДИО-Консалт', 'авиакомпания Победа'], subtopics=['Услуги компании ДИО-Консалт по автоматизации процессов', 'Опыт внедрения решений 1С в авиакомпаниях', 'Преимущества выбора компании ДИО-Консалт перед конкурентами', 'Кейс успешного сотрудничества ДИО-Консалт с крупными компаниями', "Проблематика автоматизации бизнеса авиакомпании 'Победа'"], time_frame='Актуальная информация', info_type='Исследования, аналитика, кейсы, экспертные мнения', search_strategy='Сбор информации о преимуществах услуг нашей компании и успешных примерах реализации проектов в сфере авиаперевозок.', search_queries=['опыт внедрения решений 1С в авиакомпаниях', "кейсы успешной автоматизации компаний похожих на 'Победа'", "преимущества компании 'ДИ

In [30]:
from ddgs import DDGS
from tqdm import tqdm

search_results: dict[str, list[dict[str, str]]] = {}

for search_query in tqdm(response.search_queries):
  results = DDGS().text(search_query, max_results=5)
  search_results[search_query] = results

100%|██████████| 5/5 [00:08<00:00,  1.63s/it]


In [21]:
search_results

{'преимущества внедрения 1С в авиакомпаниях': [{'title': '10 Biggest Spiders in the World - ThoughtCo',
   'href': 'https://www.thoughtco.com/biggest-spiders-in-the-world-4172117',
   'body': 'May 29, 2024 · Meet the 10 biggest spiders in the world. Get the facts about every species, then learn where to find them in the wild.'},
  {'title': '10 biggest spiders in the world: Discover gigantic, terrifying ...',
   'href': 'https://www.discoverwildlife.com/animal-facts/insects-invertebrates/biggest-spiders-in-the-world',
   'body': "Oct 23, 2025 · Meet the ten largest spiders in the world - you wouldn't want to see these scuttling across your carpet!"},
  {'title': '12 Biggest Spiders in the UK: Identify by Images and Size',
   'href': 'https://petdemy.com/biggest-spiders-in-the-uk/',
   'body': 'Jul 12, 2025 · From the massive Cardinal Spider to the semi-aquatic Raft Spider, this list highlights the 12 biggest spiders found in the UK — including their names, sizes, and where you might …'

In [35]:
RANKER_PROMPT = """Твоя задача оценить релевантность поискового запроса к его результату. Где 0 - это абсолютно нерелевантный запрос, а 1 - полностью соотвествует запросу.

**ПОИСКОВЫЙ ЗАПРОС:** {search_query}

**РЕЗУЛЬТАТ:** {search_result}

Выведи следующую информацию в формате JSON:
{format_instructions}
"""


class RankerResponse(BaseModel):
    rank: float = Field(..., description="Оценка релевантности")
    
    
ranker_parser = PydanticOutputParser(pydantic_object=RankerResponse)


ranker_chain = (
    ChatPromptTemplate
    .from_messages(["system", RANKER_PROMPT])
    .partial(format_instructions=ranker_parser.get_format_instructions())
    | giga_max
    | ranker_parser
)

In [36]:
ranked_search_results = {}

for query, results in search_results.items():
    filtered_results = []
    for result in results:
        response = ranker_chain.invoke({
            "search_query": query, 
            "search_result": result
        })
        if response.rank >= 0.5:
            filtered_results.append(result)
    
    ranked_search_results[query] = filtered_results
    
ranked_search_results

{'опыт внедрения решений 1С в авиакомпаниях': [],
 "кейсы успешной автоматизации компаний похожих на 'Победа'": [],
 "преимущества компании 'ДИО-Консалт' перед конкурентами": [],
 "автоматизация процессов авиакомпании 'Победа': проблемы и решения": [],
 "успех проектов автоматизации с участием компании 'ДИО-Консалт'": [{'title': 'ГК ДИО - Консалт - партнер 1С Тюмень',
   'href': 'https://diocon.ru/',
   'body': 'Компания " ДИО - Консалт " — официальный партнёр 1С в городе Тюмень . Мы предлагаем полный комплекс услуг: внедрение, настройка, обучение и сопровождение программ 1С. Автоматизируем бизнес-процессы, помогаем повысить эффективность и сократить...'},
  {'title': 'ГК ДИО - Консалт - партнер 1С Тюмень',
   'href': 'http://dio-it.ru/',
   'body': 'Компания " ДИО - Консалт " — официальный партнёр 1С в городе Тюмень . Мы предлагаем полный комплекс услуг: внедрение, настройка, обучение и сопровождение программ 1С.'},
  {'title': 'ДИО - Консалт - официальный партнер фирмы 1С | 1С:Центр 

In [ ]:
import requests

url = "https://api.scraperapi.com/"

payload = {
    "api_key": os.getenv("SCRAPPER_APIKEY"),
    "url": "https://elma365.com/ru/articles/6-patternov-optimizacii-biznes-processov/",
    "output_format": "markdown"
}

response = requests.get(url, params=payload)

response.text

'[![Логотип ELMA365](/assets/49289ea6-f29e-4bc5-affa-7b9ec21b3932/image.webp?format=webp "Логотип ELMA365")](/ru/)\n\n* [ Продукты](/ru/)  \n   * Low-code BPM Platform  \n    Создавайте Low-code приложения для полной автоматизации бизнеса.  \n         * [Бизнес-процессы ![arrow](/_ipx/s_16x7/svg/arrow-link-menu.svg) **Самая внедряемая** система управления бизнес‑процессами (BPM) в СНГ.](/ru/products/bpm/)  \n         * [Low-code приложения ![arrow](/_ipx/s_16x7/svg/arrow-link-menu.svg) **Сквозные процессы** на единой  BPM-платформе.](/ru/products/low-code/)  \n         * [ELMA365 Hub ![arrow](/_ipx/s_16x7/svg/arrow-link-menu.svg)  Технология комплексной  автоматизации на**Low-code платформе**.](/ru/products/hub/)  \n         * [ELMA HighLoad ![arrow](/_ipx/s_16x7/svg/arrow-link-menu.svg)  Открытый отчет для тестирования нагрузки на **10 000 пользователей.**](/ru/highload/)  \n   * Продукты на базе Low-code BPM Platform  \n    Low-code продукты обладают гибкостью настройки под особеннос